# NumPy Interoperability

PyQuantLib integrates seamlessly with NumPy for numerical computing. This notebook demonstrates conversions, zero-copy views, and practical workflows.

In [ ]:
import pyquantlib as ql
import numpy as np

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.__ql_version__})")

---

## 1. Array Conversion

PyQuantLib uses `implicitly_convertible` for `Array`: functions expecting `Array` automatically accept Python lists and numpy arrays.

In [ ]:
# All three work identically
a = [1.0, 2.0, 3.0]
b = [4.0, 5.0, 6.0]

# Explicit Array construction
result1 = ql.DotProduct(ql.Array(a), ql.Array(b))

# Pass Python lists directly (auto-converted)
result2 = ql.DotProduct(a, b)

# Pass numpy arrays directly (auto-converted)
result3 = ql.DotProduct(np.array(a), np.array(b))

print(f"Explicit Array: {result1}")
print(f"Python list:    {result2}")
print(f"NumPy array:    {result3}")

### Array → NumPy

Convert `ql.Array` to numpy with `np.array()`:

In [ ]:
arr = ql.Array([1.0, 2.0, 3.0, 4.0, 5.0])

# Convert to numpy
np_arr = np.array(arr)

print(f"Type: {type(np_arr)}")
print(f"Data: {np_arr}")
print(f"NumPy operations: mean={np_arr.mean()}, std={np_arr.std():.4f}")

---

## 2. Matrix Conversion

`ql.Matrix` is a 2-dimensional matrix. Like `Array`, it supports automatic conversion from lists and numpy arrays.

In [ ]:
# All three work identically with functions
data = [[1.0, 2.0, 3.0],
        [4.0, 5.0, 6.0]]

# Explicit Matrix construction
result1 = ql.transpose(ql.Matrix(data))

# Pass Python list directly (auto-converted)
result2 = ql.transpose(data)

# Pass numpy array directly (auto-converted)
result3 = ql.transpose(np.array(data))

print(f"All produce same result: {np.array(result1).tolist()}")

In [ ]:
# Matrix multiplication also works with auto-conversion
a = [1.0, 2.0, 3.0]
b = [4.0, 5.0]

# outerProduct accepts Array arguments - lists auto-convert
outer = ql.outerProduct(a, b)
print(f"Outer product shape: {outer.rows()}x{outer.columns()}")
print(f"Result:\n{np.array(outer)}")

In [ ]:
# Construction patterns
# Explicit construction still useful for creating Matrix objects
mat = ql.Matrix([[1.0, 0.5],
                 [0.5, 1.0]])

# Pre-allocate for incremental building
mat2 = ql.Matrix(3, 3, 0.0)  # 3x3 filled with zeros
for i in range(3):
    mat2[i, i] = 1.0  # Set diagonal

print(f"Correlation matrix:\n{mat}")
print(f"Identity matrix:\n{mat2}")

### Matrix → NumPy

In [ ]:
mat = ql.Matrix([[1, 2, 3],
                 [4, 5, 6]])

np_mat = np.array(mat)
print(f"Shape: {np_mat.shape}")
print(f"Data:\n{np_mat}")

### Row Access

Individual rows return numpy array views:

In [ ]:
mat = ql.Matrix([[10, 20, 30],
                 [40, 50, 60]])

row0 = mat[0]
row1 = mat[1]

print(f"Row 0: {row0} (type: {type(row0).__name__})")
print(f"Row 1: {row1}")
print(f"Row 0 sum: {row0.sum()}")

---

## 3. Zero-Copy Views

For performance, numpy arrays can be created that **share memory** with PyQuantLib objects (no data copying).

In [ ]:
arr = ql.Array([1.0, 2.0, 3.0, 4.0, 5.0])

# Copy (safe, independent data)
np_copy = np.array(arr)

# Zero-copy view (fast, shares memory)
np_view = np.array(arr, copy=False)

print(f"Original: {list(arr)}")
print(f"Copy:     {np_copy}")
print(f"View:     {np_view}")

### Memory Sharing Demonstration

Modifying a view affects the original:

In [ ]:
arr = ql.Array([1.0, 2.0, 3.0])
view = np.array(arr, copy=False)

print(f"Before: arr={list(arr)}, view={view}")

# Modify through the view
view[0] = 999.0

print(f"After:  arr={list(arr)}, view={view}")

### Performance Comparison

In [ ]:
import time

# Large array
large_arr = ql.Array(list(range(1_000_000)))

# Benchmark copy
start = time.perf_counter()
for _ in range(100):
    np_copy = np.array(large_arr)
copy_time = time.perf_counter() - start

# Benchmark view
start = time.perf_counter()
for _ in range(100):
    np_view = np.array(large_arr, copy=False)
view_time = time.perf_counter() - start

print(f"Array size: {len(large_arr):,} elements")
print(f"Copy (100x): {copy_time*1000:.1f} ms")
print(f"View (100x): {view_time*1000:.1f} ms")
print(f"Speedup: {copy_time/view_time:.0f}x")

---

## 4. Practical Example: Volatility Surface

Build a Black variance surface from numpy data and extract it for analysis.

In [ ]:
# Setup
today = ql.Date(15, 6, 2025)
ql.Settings.instance().evaluationDate = today
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
day_counter = ql.Actual365Fixed()

In [ ]:
# Market data as numpy arrays
strikes = np.array([80, 90, 100, 110, 120], dtype=float)
tenors = np.array([0.25, 0.5, 1.0, 2.0])  # years

# Implied volatilities (strikes x tenors)
vols = np.array([
    [0.28, 0.26, 0.24, 0.22],  # K=80
    [0.24, 0.23, 0.22, 0.21],  # K=90
    [0.22, 0.21, 0.20, 0.19],  # K=100 (ATM)
    [0.24, 0.23, 0.22, 0.21],  # K=110
    [0.28, 0.26, 0.24, 0.22],  # K=120
])

print(f"Strikes: {strikes}")
print(f"Tenors: {tenors}")
print(f"Vol matrix shape: {vols.shape}")

In [ ]:
# Convert tenors to QuantLib dates
expiries = [calendar.advance(today, ql.Period(int(t * 12), ql.Months)) for t in tenors]

# Build BlackVarianceSurface (expects Matrix)
vol_matrix = ql.Matrix(vols)

surface = ql.BlackVarianceSurface(
    today,
    calendar,
    expiries,
    list(strikes),
    vol_matrix,
    day_counter
)

print(f"Surface created: {type(surface).__name__}")

In [ ]:
# Query the surface
test_strikes = [85, 100, 115]
test_tenors = [0.25, 1.0]

print("Interpolated volatilities:")
print(f"{'Strike':>8} {'T=0.25':>8} {'T=1.0':>8}")
print("-" * 26)
for k in test_strikes:
    vol_025 = surface.blackVol(0.25, k)
    vol_1 = surface.blackVol(1.0, k)
    print(f"{k:>8} {vol_025:>8.2%} {vol_1:>8.2%}")

### Extract Surface to Pandas

In [ ]:
import pandas as pd

# Build a dense grid
grid_strikes = np.linspace(80, 120, 9)
grid_tenors = np.array([0.25, 0.5, 0.75, 1.0, 1.5, 2.0])

# Extract vols
vol_grid = np.array([
    [surface.blackVol(t, k) for t in grid_tenors]
    for k in grid_strikes
])

# Create DataFrame
df = pd.DataFrame(
    vol_grid,
    index=grid_strikes,
    columns=[f"{t}Y" for t in grid_tenors]
)
df.index.name = "Strike"

print("Volatility Surface (extracted to pandas):")
df.style.format("{:.2%}")

---

## 5. Performance Tips

| Scenario | Recommendation |
|----------|----------------|
| Reading data, source stays alive | Use `copy=False` for zero-copy view |
| Source may be modified/destroyed | Use default copy |
| Functions expecting `Array` | Pass lists/numpy directly (auto-converted) |
| Functions expecting `Matrix` | Pass lists/numpy directly (auto-converted) |
| Building matrices incrementally | Pre-allocate with `ql.Matrix(rows, cols)` |
| Large arrays in loops | Reuse `ql.Array` objects to avoid repeated copies |

### How It Works

PyQuantLib uses two pybind11 mechanisms:

| Direction | Mechanism | Cost |
|-----------|-----------|------|
| QuantLib → NumPy | Buffer protocol | Zero-copy (shared memory) |
| Python → QuantLib | Implicit conversion | Copy (new object created) |

Zero-copy views share memory (modifications affect both sides). Automatic conversion creates temporary copies. For performance-critical code with large arrays, create `ql.Array` objects once and reuse them.

In [ ]:
# Tip: Pre-allocation benchmark
n = 500

# Method 1: Build numpy then convert
start = time.perf_counter()
np_mat = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        np_mat[i, j] = i * j
mat1 = ql.Matrix(np_mat)
method1_time = time.perf_counter() - start

# Method 2: Pre-allocate ql.Matrix
start = time.perf_counter()
mat2 = ql.Matrix(n, n)
for i in range(n):
    for j in range(n):
        mat2[i, j] = i * j
method2_time = time.perf_counter() - start

print(f"Build numpy then convert: {method1_time*1000:.1f} ms")
print(f"Pre-allocate ql.Matrix:   {method2_time*1000:.1f} ms")
print(f"Winner: {'numpy' if method1_time < method2_time else 'pre-allocate'}")

---

## Summary

| Type | Python → QuantLib | QuantLib → NumPy |
|------|-------------------|------------------|
| Array | Automatic (list, numpy) | `np.array(arr)` or `np.array(arr, copy=False)` |
| Matrix | Automatic (list of lists, numpy) | `np.array(mat)` or `np.array(mat, copy=False)` |

See also: [NumPy Documentation](../docs/numpy.md)